<a href="https://colab.research.google.com/github/chuviethongfpt/atom-assignments/blob/main/Project_Report_28062021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os
import numpy as np

In [2]:
!ls

drive  project-cloud-function-316413-e8f7e070f21f.json	sample_data


In [3]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'project-cloud-function-316413-e8f7e070f21f.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


In [4]:
#Get data from Google sheet
spreadsheet_key = '1kfakqgkjVmyHlgZfFIqitLi-A3lUr4KtZJUTwpXqV54' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.worksheet('customers') #-> 0 - first sheet, 1 - second sheet etc. 
customers = pd.DataFrame(worksheet.get_all_records())
customers #sheet 1

,CustomersId,Firstname,Lastname,Country,City,Phone
0,1,Maria,Anders,Bình Dương,Dĩ An,030-0074321
1,2,Ana,Trujillo,Bình Dương,Dĩ An,(5) 555-4729
2,3,Antonio,Moreno,Bình Dương,Dĩ An,(5) 555-3932
3,4,Thomas,Hardy,Bình Dương,Dĩ An,(171) 555-7788
4,5,Christina,Berglund,Bình Dương,Dĩ An,0921-12 34 65
...,...,...,...,...,...,...
86,87,Pirkko,Koskitalo,Bình Dương,Dĩ An,981-443655
87,88,Paula,Parente,Bình Dương,Dĩ An,(14) 555-8122
88,89,Karl,Jablonski,Bình Dương,Dĩ An,(206) 555-4112
89,90,Matti,Karttunen,Bình Dương,Dĩ An,90-224 8858


In [5]:
spreadsheet_key = '1kfakqgkjVmyHlgZfFIqitLi-A3lUr4KtZJUTwpXqV54' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.worksheet('products') #-> 0 - first sheet, 1 - second sheet etc. 
products = pd.DataFrame(worksheet.get_all_records())
products[['UnitePrice']] = products[['UnitPrice']].astype(float)
products #sheet 2

,Id_products,ProductName,SupplierId,UnitPrice,Package,IsDiscontinued,UnitePrice
0,1,Gạch tàu 30x30,1,70,1 boxes x 11 tablets,0,70.0
1,2,gạch lát nền 40x40,1,72,1 boxes x 6 tablets,0,72.0
2,3,gạch men 200x200mm,1,90,1 boxes x 6 tablets,0,90.0
3,4,gạch men 250x250mm,2,95,1 boxes x 6 tablets,0,95.0
4,5,gạch men 300x300mm,2,80,1 boxes x 4 tablets,1,80.0
...,...,...,...,...,...,...,...
72,73,Sơn Jotun Jotashield,22,5994,15 liter,0,5994.0
73,74,Sơn Jotun Essence,22,1143,5 liter,0,1143.0
74,75,Sơn Jotun Essence,22,3643,17 liter,0,3643.0
75,76,Sơn Jotun Majestic,22,1446,5 liter,0,1446.0


In [6]:
spreadsheet_key = '1kfakqgkjVmyHlgZfFIqitLi-A3lUr4KtZJUTwpXqV54' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.worksheet('orders') #-> 0 - first sheet, 1 - second sheet etc. 
orders = pd.DataFrame(worksheet.get_all_records())
orders[['TotalAmount']] = orders[['TotalAmount']].astype(float)
orders['OrderDate'] = pd.to_datetime(orders['OrderDate'])
orders #sheet 3

,Id_orders,OrderDate,OrderNumber,CustomersId,TotalAmount
0,1,2012-04-07,542378,85,440.0
1,2,2012-05-07,542379,79,1863.0
2,3,2012-08-07,542380,34,1813.0
3,4,2012-08-07,542381,84,671.0
4,5,2012-09-07,542382,76,3730.0
...,...,...,...,...,...
825,826,2014-05-05,543203,58,300.0
826,827,2014-06-05,543204,73,244.0
827,828,2014-06-05,543205,68,586.0
828,829,2014-06-05,543206,9,1057.0


In [7]:
spreadsheet_key = '1kfakqgkjVmyHlgZfFIqitLi-A3lUr4KtZJUTwpXqV54' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.worksheet('orderitems') #-> 0 - first sheet, 1 - second sheet etc. 
orderitems = pd.DataFrame(worksheet.get_all_records())
orderitems #sheet 4

,Id_items,OrderId,ProductId,UnitPrice,Quantity
0,1,1,11,150,12
1,2,1,42,1439,10
2,3,1,72,2033,5
3,4,2,14,85,9
4,5,2,51,4329,40
...,...,...,...,...,...
2150,2151,830,64,560,2
2151,2152,830,66,4550,1
2152,2153,830,73,5994,2
2153,2154,830,75,3643,4


In [8]:
spreadsheet_key = '1kfakqgkjVmyHlgZfFIqitLi-A3lUr4KtZJUTwpXqV54' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.worksheet('suppliers') #-> 0 - first sheet, 1 - second sheet etc. 
suppliers = pd.DataFrame(worksheet.get_all_records())
suppliers

,Id_suppliers,CompanyName,ContactName,ContactTitle,City,Country,Phone
0,1,Công ty đầu tư và xây dựng Đại Đồng Đen,Trần Nhật Quang,NULL,Hồ Chí Minh,Việt Nam,(171) 555-2222
1,2,Công ty TNHH Vật liệu và Xây dựng Bình Dương,Hồ Bích Lâm,NULL,Hồ Chí Minh,Việt Nam,(100) 555-4822
2,3,Công Ty TNHH Thương Mại Dịch Vụ CVP (Gạch Bông...,Nguyễn Quốc Thịnh,NULL,Hồ Chí Minh,Việt Nam,(313) 555-5735
3,4,Gạch block Trung Phương Nam,Đỗ Tú Tài,NULL,Bình Dương,Việt Nam,(03) 3555-5011
4,5,Công ty TNHH Gạch Bông Việt Nam,Trần Thế Cường,NULL,Bình Dương,Việt Nam,(98) 598 76 54
5,6,Công Ty Cp Trang Trí Nội Thất - Xây Dựng - Đầu...,Nguyễn Văn Hậu,NULL,Hồ Chí Minh,Việt Nam,(06) 431-7877
6,7,CT Xi măng Nghi Sơn - CN TP.HCM,Ian Devling,NULL,Hồ Chí Minh,Việt Nam,(03) 444-2343
7,8,Công Ty CP Xi Măng Vicem Hà Tiên,Nguyễn Tiến Dũng,NULL,Bình Dương,Việt Nam,(161) 555-4448
8,9,Công Ty TNHH SAKO Việt Nam,Lê Bích Trâm,NULL,Thủ Đức,Việt Nam,031-987 65 43
9,10,CÔNG TY TNHH MTV TRANG TRÍ NỘI THẤT NAM VIỆT,Đỗ Thị Huế,NULL,Hồ Chí Minh,Việt Nam,(11) 555 4640


In [9]:
!pip install duckdb

     |████████████████████████████████| 9.0MB 5.2MB/s 


In [10]:
# to install: pip install duckdb
import pandas as pd
import duckdb

In [11]:
con = duckdb.connect(database=':memory:', read_only=False) # -> Tạo DB connection
# create a table
con.execute("SELECT * FROM customers LIMIT 10").fetch_df() 

,CustomersId,Firstname,Lastname,Country,City,Phone
0,1,Maria,Anders,Bình Dương,Dĩ An,030-0074321
1,2,Ana,Trujillo,Bình Dương,Dĩ An,(5) 555-4729
2,3,Antonio,Moreno,Bình Dương,Dĩ An,(5) 555-3932
3,4,Thomas,Hardy,Bình Dương,Dĩ An,(171) 555-7788
4,5,Christina,Berglund,Bình Dương,Dĩ An,0921-12 34 65
5,6,Hanna,Moos,Bình Dương,Dĩ An,0621-08460
6,7,Fradacrique,Citeaux,Bình Dương,Dĩ An,88.60.15.31
7,8,MartÃ­n,Sommer,Bình Dương,Dĩ An,(91) 555 22 82
8,9,Laurence,Lebihan,Bình Dương,Thuận An,91.24.45.40
9,10,Elizabeth,Lincoln,Bình Dương,Thuận An,(604) 555-4729


In [12]:
#Liệt kê khoảng 10, 20 khách hàng có đơn đặt hàng nhiều nhất 
#WHERE WHERE MONTH(o.OrderDate)=3 AND YEAR(o.OrderDate)=2014
SQL = '''
  SELECT 
    CONCAT(c.Firstname,' ',c.Lastname) AS customer_name,
    c.Country, 
    c.City, 
    COUNT(o.Id_orders) AS count_orders 
  FROM customers c LEFT JOIN orders o ON c.CustomersId=o.CustomersId
  GROUP BY customer_name, c.Country, c.City
  ORDER BY count_orders DESC
  LIMIT 20
'''
con.execute(SQL).fetch_df()

,customer_name,Country,City,count_orders
0,Jose Pavarotti,Bình Dương,Dĩ An,31
1,Roland Mendel,Bình Dương,Dĩ An,30
2,Horst Kloss,Bình Dương,Lái Thiêu,28
3,Maria Larsson,Bình Dương,Thủ Dầu Một,19
4,Patricia McKenna,Bình Dương,Thuận An,19
5,Christina Berglund,Bình Dương,Dĩ An,18
6,Carlos Hernaindez,Bình Dương,Thuận An,18
7,Paula Wilson,Bình Dương,Dĩ An,18
8,Laurence Lebihan,Bình Dương,Thuận An,17
9,Peter Franken,Bình Dương,Thủ Dầu Một,15


In [13]:
#Liệt kê 20 đơn đặt hàng có doanh thu cao nhất từ trước đến nay 
SQL = '''
SELECT * 
FROM orders
WHERE Id_orders IN
(SELECT orderid FROM orderitems WHERE ProductId IN 
 (SELECT Id_products FROM products))
ORDER BY TotalAmount DESC
LIMIT 20
'''
con.execute(SQL).fetch_df()

,Id_orders,OrderDate,OrderNumber,CustomersId,TotalAmount
0,618,2014-02-02,542995,63,17250.0
1,783,2014-04-17,543160,71,16322.0
2,734,2014-03-27,543111,34,15810.0
3,125,2012-04-12,542502,62,12281.0
4,177,2013-01-23,542554,51,11493.0
5,570,2014-06-01,542947,39,11491.0
6,642,2014-02-16,543019,65,11380.0
7,170,2013-01-16,542547,73,11283.0
8,650,2014-02-19,543027,37,10835.0
9,106,2012-11-13,542483,59,10742.0


In [14]:
#Thống kê products có số lượng đặt hàng nhiều nhất trong quý hoặc trong năm, gồm các thông tin trong bảng như : id, productname, sum(Quantity), AVG(Unitprice)
SQL = '''
SELECT p.Id_products, p.ProductName, SUM(i.Quantity) AS sum_quantity, AVG(p.UnitPrice) AS avg_unitprice
FROM products p LEFT JOIN orderitems i ON p.Id_products=i.ProductId LEFT JOIN orders o ON o.Id_orders=i.OrderId
WHERE p.IsDiscontinued=0
GROUP BY p.Id_products,p.ProductName
ORDER BY sum_quantity DESC
'''
con.execute(SQL).fetch_df()


,Id_products,ProductName,sum_quantity,avg_unitprice
0,60,Sơn EcoPlus A66,1577.0,980.0
1,59,Sơn Dulux Y65,1496.0,4215.0
2,31,Sơn chống gỉ loại 1,1397.0,1120.0
3,56,Sơn Dulux Y65,1263.0,225.0
4,16,Xi măng Nghi Sơn,1158.0,75.0
...,...,...,...,...
65,50,Sơn Dulux 79A,235.0,3760.0
66,67,Sơn EcoPlus A96,184.0,5110.0
67,48,Sơn Dulux 79A,138.0,2980.0
68,37,Sơn TOA loại 2,125.0,2153.0


In [15]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")

Setup Complete


In [16]:
df=pd.merge(customers, orders, how='left', on='CustomersId')

In [17]:
df1=df.groupby('Firstname')['CustomersId'].count().sort_values(ascending=False).head(10)
df1

Firstname
Carlos       32
Jose         31
Roland       30
Horst        28
Paula        27
Maria        25
Patricia     19
Christina    18
Elizabeth    17
Laurence     17
Name: CustomersId, dtype: int64

In [18]:
SQL1 = '''
SELECT 
  YEAR(OrderDate) AS year_order, 
  COUNT(*) AS count_order, 
  SUM(totalamount) AS sum_totalamount
FROM orders
WHERE YEAR(OrderDate) BETWEEN 2012 AND 2014
GROUP BY YEAR(orderdate)
ORDER BY YEAR(orderdate)
'''
con.execute(SQL1).fetch_df()

,year_order,count_order,sum_totalamount
0,2012,152,226302.0
1,2013,408,658409.0
2,2014,270,469790.0


In [21]:
orders['year'] = pd.DatetimeIndex(orders['OrderDate']).year
orders.groupby(['year']).sum({"TotalAmount": [np.size, np.mean]})
orders

,Id_orders,OrderDate,OrderNumber,CustomersId,TotalAmount,year
0,1,2012-04-07,542378,85,440.0,2012
1,2,2012-05-07,542379,79,1863.0,2012
2,3,2012-08-07,542380,34,1813.0,2012
3,4,2012-08-07,542381,84,671.0,2012
4,5,2012-09-07,542382,76,3730.0,2012
...,...,...,...,...,...,...
825,826,2014-05-05,543203,58,300.0,2014
826,827,2014-06-05,543204,73,244.0,2014
827,828,2014-06-05,543205,68,586.0,2014
828,829,2014-06-05,543206,9,1057.0,2014


In [27]:
list_of_years = orders['OrderDate']
df = pd.DataFrame({'OrderDate': pd.to_datetime(list_of_years)})
L = ['year']
df = df.join(pd.concat((getattr(df['OrderDate'].dt, i).rename(i) for i in L), axis=1))
df

,OrderDate,year
0,2012-04-07,2012
1,2012-05-07,2012
2,2012-08-07,2012
3,2012-08-07,2012
4,2012-09-07,2012
...,...,...
825,2014-05-05,2014
826,2014-06-05,2014
827,2014-06-05,2014
828,2014-06-05,2014


In [22]:
dff=orders.groupby('year')["TotalAmount"].sum({"OrderNumber": np.size})
df1=pd.DataFrame(dff)
df1

,TotalAmount
year,
2012,226302.0
2013,658409.0
2014,469790.0


In [23]:
dff2 = orders.groupby('year')['OrderNumber'].count() 
df2=pd.DataFrame(dff2)
df2

,OrderNumber
year,
2012,152
2013,408
2014,270


In [24]:
df3=pd.merge(df1, df2, on='year')
df3

,TotalAmount,OrderNumber
year,,
2012,226302.0,152
2013,658409.0,408
2014,469790.0,270


In [ ]:
plt.figure(figsize=(10,6)) # Your code here
plt.title("Average score for racing games by platform")
sns.barplot(x=dff['year'], y=dff.index)
plt.ylabel("Average score")

In [ ]:
orders.groupby(["year"]).sum({"TotalAmount": [np.size, np.mean]})

In [28]:
!pip install streamlit

     |████████████████████████████████| 7.8MB 5.2MB/s 
     |████████████████████████████████| 112kB 49.1MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
     |████████████████████████████████| 4.2MB 43.0MB/s 
     |████████████████████████████████| 174kB 43.9MB/s 
     |████████████████████████████████| 122kB 38.4MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13476 sha256=b6426e5f9c1cfe570fff0129a564faa8c6b135517b232a53d388830ce77f0d32
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.5 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


In [29]:
!pip install ipython

In [31]:
!pip install pyngrok

     |████████████████████████████████| 747kB 6.7MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.0.5-cp37-none-any.whl size=19262 sha256=df6d22a2c10940586e521c2a6ab95c63daa2bbcfb054279277c7beccf0ce186f
  Stored in directory: /root/.cache/pip/wheels/0c/13/64/5ebbcc22eaf53fdf5766b397c1fb17c83f5775fdccf0ea1b88
Successfully built pyngrok


In [32]:
!pip install ipykernel